pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [ ]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2 MB 34 kB/s 
     |████████████████████████████████| 198 kB 58.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=1cd8d89bd9440129924086cc7ac0deb7a284f57d2c1d9e6957d82cfe1d3e2fdd
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Taipei Housing Price Prediction") \
    .getOrCreate()

# 타이베이 주택 가격 예측 모델 만들기




데이터셋 설명

이번 문제는 대만 타이베이 시의 신단 지역에서 수집된 주택 거래 관련 정보를 바탕으로 주택 가격(정확히는 주택의 평당 가격)을 예측하는 Regression 모델을 만들어보는 것이다. 총 6개의 피쳐와 주택의 평당 가격에 해당하는 레이블 정보가 훈련 데이터로 제공된다. 레이블의 경우에는 주택의 최종 가격이 아니라 평당 가격이란 점을 다시 한번 강조한다.

각 컬럼에 대한 설명은 아래와 같으며 모든 필드는 X4를 제외하고는 실수 타입이다.

* X1: 주택 거래 날짜를 실수로 제공한다. 소수점 부분은 달을 나타낸다. 예를 들어 2013.250이라면 2013년 3월임을 나타낸다 (0.250 = 3/12)
* X2: 주택 나이 (년수)
* X3: 가장 가까운 지하철역까지의 거리 (미터)
* X4: 주택 근방 걸어갈 수 있는 거리내 편의점 수
* X5: 주택 위치의 위도 (latitude)
* X6: 주택 위치의 경도 (longitude)
* Y: 주택 평당 가격



In [ ]:
spark

In [ ]:
!wget https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/Taipei_sindan_housing.csv

--2022-03-08 10:38:30--  https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/Taipei_sindan_housing.csv
Resolving grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)... 52.219.146.15
Connecting to grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)|52.219.146.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20014 (20K) [text/csv]
Saving to: ‘Taipei_sindan_housing.csv’

Taipei_sindan_housi 100%[===================>]  19.54K   123KB/s    in 0.2s    

2022-03-08 10:38:31 (123 KB/s) - ‘Taipei_sindan_housing.csv’ saved [20014/20014]



In [ ]:
!ls -tl

total 24
drwxr-xr-x 1 root root  4096 Feb 18 14:33 sample_data
-rw-r--r-- 1 root root 20014 Jul 17  2021 Taipei_sindan_housing.csv


In [ ]:
data = spark.read.csv('./Taipei_sindan_housing.csv', header=True, inferSchema=True)

In [ ]:
data.printSchema()

root
 |-- X1: double (nullable = true)
 |-- X2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- X4: integer (nullable = true)
 |-- X5: double (nullable = true)
 |-- X6: double (nullable = true)
 |-- Y: double (nullable = true)



In [ ]:
data.show()

+--------+----+--------+---+--------+---------+----+
|      X1|  X2|      X3| X4|      X5|       X6|   Y|
+--------+----+--------+---+--------+---------+----+
|2012.917|32.0|84.87882| 10|24.98298|121.54024|37.9|
|2012.917|19.5|306.5947|  9|24.98034|121.53951|42.2|
|2013.583|13.3|561.9845|  5|24.98746|121.54391|47.3|
|  2013.5|13.3|561.9845|  5|24.98746|121.54391|54.8|
|2012.833| 5.0|390.5684|  5|24.97937|121.54245|43.1|
|2012.667| 7.1| 2175.03|  3|24.96305|121.51254|32.1|
|2012.667|34.5|623.4731|  7|24.97933|121.53642|40.3|
|2013.417|20.3|287.6025|  6|24.98042|121.54228|46.7|
|  2013.5|31.7|5512.038|  1|24.95095|121.48458|18.8|
|2013.417|17.9| 1783.18|  3|24.96731|121.51486|22.1|
|2013.083|34.8|405.2134|  1|24.97349|121.53372|41.4|
|2013.333| 6.3|90.45606|  9|24.97433| 121.5431|58.1|
|2012.917|13.0|492.2313|  5|24.96515|121.53737|39.3|
|2012.667|20.4|2469.645|  4|24.96108|121.51046|23.8|
|  2013.5|13.2|1164.838|  4|24.99156|121.53406|34.3|
|2013.583|35.7|579.2083|  2| 24.9824|121.54619

In [ ]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+--------------------+--------------------+------------------+
|summary|                X1|                X2|                X3|                X4|                  X5|                  X6|                 Y|
+-------+------------------+------------------+------------------+------------------+--------------------+--------------------+------------------+
|  count|               414|               414|               414|               414|                 414|                 414|               414|
|   mean|2013.1489710144933| 17.71256038647343|1083.8856889130436| 4.094202898550725|  24.969030072463745|  121.53336108695667| 37.98019323671498|
| stddev|0.2819672402629999|11.392484533242524| 1262.109595407851|2.9455618056636177|0.012410196590450208|0.015347183004592374|13.606487697735316|
|    min|          2012.667|               0.0|          23.38284|                 0|            24.93207|           1

In [ ]:
# get the dimensions of the data
(data.count() , len(data.columns))

(414, 7)

In [ ]:
import pyspark.sql.functions as f

# null values in each column
data_agg = data.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in data.columns])
data_agg.show()

+---+---+---+---+---+---+---+
| X1| X2| X3| X4| X5| X6|  Y|
+---+---+---+---+---+---+---+
|  0|  0|  0|  0|  0|  0|  0|
+---+---+---+---+---+---+---+



### 데이터를 어떻게 요리할 지 생각 해보자. 
- X1: 주택 거래 날짜를 실수로 제공한다. 소수점 부분은 달을 나타낸다. 예를 들어 2013.250이라면 - 2013년 3월임을 나타낸다 (0.250 = 3/12)
- X2: 주택 나이 (년수)
- X3: 가장 가까운 지하철역까지의 거리 (미터)
- X4: 주택 근방 걸어갈 수 있는 거리내 편의점 수
- X5: 주택 위치의 위도 (latitude)
- X6: 주택 위치의 경도 (longitude)
- Y: 주택 평당 가격

1. X1, X2, X3 min-max scaling
2. X5, X6 standardzation 후 제곱형태로 구해 카이제곱의 분포 형태로 바꿔준다.\
 이유: 중심에서 얼마나 벗어 나있는지에 따라 집값이 많이 차이 날꺼 같음
3. X4 는 정수 타입으로 그냥 냅두자. 
4. 결측치가 존재 하지 않는다.
x1, x2, x3 minmax sclaing > x5, x6 정규화 후 제곱 형태로 치환 >vector assembler > linear Regression > 예측




In [ ]:
from pyspark.ml import feature
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

assembler1 = VectorAssembler(inputCols=['X1', 'X2', 'X3'], outputCol='feature1')
assembler2 = VectorAssembler(inputCols=['X5', 'X6'], outputCol='feature2')

mmscaler = MinMaxScaler(inputCol='feature1', outputCol="adjusted_f1")
stdscaler = StandardScaler(inputCol='feature2', outputCol="std_feature")
# power = ElementwiseProduct(scalingVec=, outputCol='adjusted_f2') # 이 피쳐를 제곱하는 법을 잘 모르겠음 해당 방법 탐구 해야할 듯

assembler = VectorAssembler(inputCols=['adjusted_f1', 'X4', 'std_feature'], outputCol='Features')
reg = LinearRegression(featuresCol='Features', labelCol="Y")

pipeline = Pipeline(stages=[assembler1, assembler2, mmscaler, stdscaler, assembler, reg])


In [ ]:
train, test = data.randomSplit([0.7, 0.3])

In [ ]:
model = pipeline.fit(train)

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.select("Y", 'prediction').show()

+----+------------------+
|   Y|        prediction|
+----+------------------+
|51.8| 48.00669536880676|
|49.7|  47.8756115813724|
|50.0| 49.65854960871573|
|42.5| 39.89298388625619|
|37.4| 40.00205953474622|
|22.8|28.225813930536134|
|23.8|  27.2517025034106|
|15.5| 9.554121739364092|
|13.2| 9.541769754930357|
|53.3| 42.92095130460737|
|39.7| 39.49537354126849|
|44.0|  44.2041191925812|
|39.3|43.254449061492124|
|55.1| 47.11196638707705|
|11.6|13.243160292831163|
|25.0| 40.20157105478327|
|35.7|30.259887454107485|
|71.0| 48.92575277794393|
|27.0|31.444160450840428|
|33.4|34.897792768085566|
+----+------------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Y", metricName='rmse')
evaluator.evaluate(predictions)

8.512453701189763